# load model

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from toolkit.nlp import NLPTrainingConfig
path = './outputs/optimal_checkpoint_epoch2/'
config = NLPTrainingConfig.load(path)

model = AutoModelForCausalLM.from_pretrained(path, trust_remote_code=True, torch_dtype='auto')
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True)

/usr/local/python3.11.2/lib/python3.11/site-packages/torchmetrics/utilities/imports.py:24: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _PYTHON_LOWER_3_8 = LooseVersion(_PYTHON_VERSION) < LooseVersion("3.8")


2023-10-18 15:08:23,253 <INFO> Configuration: 👻 Custom attributes:
2023-10-18 15:08:23,254 <INFO> Configuration:    cache_dir=None
2023-10-18 15:08:23,254 <INFO> Configuration:    model_revision=main
2023-10-18 15:08:23,255 <INFO> Configuration:    use_fast_tokenizer=True
2023-10-18 15:08:23,256 <INFO> Configuration:    use_auth_token=False
2023-10-18 15:08:23,256 <INFO> Configuration:    preprocessing_num_workers=8
2023-10-18 15:08:23,257 <INFO> Configuration:    max_seq_length=2048
2023-10-18 15:08:23,257 <INFO> Configuration:    deepspeed_config=./ds_zero3_offload.hjson
2023-10-18 15:08:23,258 <INFO> Configuration:    generate_config_file=generate_config.json
2023-10-18 15:08:23,258 <INFO> Configuration:    re_gen_num=2
2023-10-18 15:08:23,259 <INFO> Configuration:    total_steps_num=470
2023-10-18 15:08:23,260 <INFO> Configuration:    steps_per_epoch=94


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
import pandas as pd
df = pd.read_json(config.val_file_path, lines=True)
df.head(2)

,instruction,input,output,query
0,,徐怀钰参加综艺节目乘风后，引起了广泛关注。她曾经经历过家庭贫困和爷爷的心理问题，复出时效果并...,\n徐怀钰近期参加综艺节目《乘风》复出，享受舞台找回自我。\n徐怀钰的经历充满波折，家庭贫困...,None
1,,南京同曦官宣引进法尔和布莱克尼 法尔和布莱克尼加入球队。随着南京同曦更换赞助商之后，球队在引...,同曦男篮宣布引进法尔和布莱克尼，给球队的阵容注入了新的活力。他们的加入有望帮助球队突破历史，...,媒体人谈同曦新双外：针对性很强


In [4]:
# import pandas as pd
# df = pd.read_json("data/hot_finetune_data/train/all.json", lines=True)
# df.head(2)

In [19]:
from build_dataset import prompt_transfer
import torch
torch.cuda.set_device(0)

model.cuda()
for idx, row in df.iterrows():
    inputs = prompt_transfer(row['input'], row['output'])
    print(inputs)
    batch = tokenizer(inputs, return_tensors='pt')
    batch = {key: value.cuda() for key, value in batch.items()}
    outputs = model.generate(**batch, **config.generate_kwargs)
    texts = []
    for idx, output in enumerate(outputs):
        texts.append(tokenizer.decode(output[batch['input_ids'][idx].size(0):], skip_special_tokens=True))
    # texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    break

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
给定文本：
徐怀钰参加综艺节目乘风后，引起了广泛关注。她曾经经历过家庭贫困和爷爷的心理问题，复出时效果并不理想。然而，她在乘风中逐渐找回了自我，与秋瓷炫等人成为了好友，并和黄丽玲合作。虽然她不是王心凌，但乘风提供了一个机会，让她重新找回自我，比输赢更重要。徐怀钰的经历令人心疼。她从小家庭贫困，爷爷对她不好，留下了永远的心理创伤。这些年，她虽然一直在努力复出，但效果都不好。在参加乘风时，一公因路灯下的小姑娘被抨击而难过。好在Ella等人一起安慰她，她才挺过来。五公时，慢热的徐怀钰终于找到了感觉。秋瓷炫都为徐怀钰的变化感觉高兴，看得出她开始享受舞台了。这次居然主动和黄丽玲合作，要知道，这两位实力唱将合作，应该会很有很多火花。黄丽玲也很期待。徐怀钰不是王心凌，但她的复出并不仅仅是为了赢。对于粉丝来说，乘风只是一个综艺，能让徐怀钰找回自我，比什么输赢都重要。乘风让徐怀钰逐渐敞开心扉，与秋瓷炫等人成为了好友，也让她重新找到了表演的乐趣。她还和黄丽玲合作过，黄丽玲也很期待。有些人可能对徐怀钰的表现有所期待，毕竟她曾经是一位著名歌手，但输赢对她来说似乎已经不再重要了。
生成300字左右视频文案。

### Response: 


In [22]:
print(texts[0])


徐怀钰参加综艺节目《乘风》引发关注，她曾是家庭贫困和爷爷心理问题的受害者，复出效果不佳。
徐怀钰从小家庭贫困，爷爷对她不好，留下了永远的心理创伤。虽然她一直努力复出，但效果都不好。参加《乘风》时，一公因路灯下的小姑娘被抨击而难过，好在Ella等人安慰她，她才挺过来。五公时，慢热的徐怀钰终于找到了感觉，秋瓷炫都为她变化感到高兴，她开始享受舞台了。甚至主动与黄丽玲合作，要知道，这两位实力唱将合作，应该会很有很多火花。黄丽玲也很期待。
徐怀钰虽然不是王心凌，但她的复出并不仅仅是为了赢。对于粉丝来说，《乘风》只是一个综艺，能让徐怀钰找回自我，比什么输赢都重要。


In [21]:
print(row['output'])


徐怀钰近期参加综艺节目《乘风》复出，享受舞台找回自我。
徐怀钰的经历充满波折，家庭贫困、爷爷的心理问题，这些年一直在努力付出。虽然在《乘风》的舞台上复出效果不理想，但我们也看到了徐怀钰逐渐找回自我，这比比赛的输赢更重要。在《乘风》中，她与秋瓷炫、黄丽玲等人成为了好友，并携手合作。虽然她没有王心凌的机遇，但《乘风》也为她提供了一个重要的机会。她因一公被抨击而感到难过，好在有Ella等人的安慰与支持，让她挺过了难关。五公时，慢热的徐怀钰终于找到了感觉，秋瓷炫也为她的变化感到高兴，看出她开始享受舞台了。这次，徐怀钰主动和黄丽玲合作，无疑将带来许多火花。
作为徐怀钰复出的舞台，《乘风》让徐怀钰找回终于找回了自我。让我们一起期待徐怀钰更多精彩表现！


# test evaluator

In [ ]:
from toolkit.training import Evaluator
from load_data_fn import load_data_fn
from toolkit.enums import Split
from toolkit.nlp import TextDataset
from toolkit.metric import rouge, bleu
def eval_callback(all_labels, all_logits, mean_loss):
    metric = rouge(all_logits, all_labels, "zh", ("rouge1", "rouge2", "rougeL"))
    metric.update(bleu(all_logits, all_label, 'zh', ("bleu1", "bleu2", "bleu3", "bleu4")))
    return metric, all_labels, all_logits

config.infer_batch_size=8
dataset = TextDataset.from_file(config.val_file_path, tokenizer, Split.VALIDATION, config, load_data_fn, use_cache=False, debug=True)
evaluator = Evaluator('generate', Split.VALIDATION, config, model, dataset, eval_callback, tokenizer=tokenizer, extral_args_evaluation=None)
metric, all_label, all_logits = evaluator.eval(1)

In [ ]:
print(metric)
for i in range(8):
    print("-----label------")
    print(all_label[i])
    print("-----pred------")
    print(all_logits[i])
    print('########################################')

# test extral evaluator

In [ ]:
# import json

# with open("generate_config.json", "r", encoding='utf8') as f:
#     generate_config = json.load(f)


# with open("generate_config.json", "w", encoding='utf8') as f:
#     json.dump(generate_config, f, indent=2)

# generate_config


In [4]:
from extral_evaluator import Extral_Evaluator, extra_calculate_metric_callback
from load_data_fn import load_data_fn
from toolkit.enums import Split
from toolkit.nlp import TextDataset
from toolkit.metric import self_bleu
def extra_calculate_metric_callback(all_labels, all_logits):
    return self_bleu(all_logits, 'zh', ("bleu1", "bleu2", "bleu3", "bleu4"), smoothing_level=1), all_labels, all_logits

# config.generate_config_file="generate_config.json"
# config.re_gen_num=2
config.infer_batch_size=8


dataset = TextDataset.from_file(config.val_file_path, tokenizer, Split.VALIDATION, config, load_data_fn, use_cache=False, debug=True)
evaluator = Extral_Evaluator('generate', Split.VALIDATION, config, model, dataset, extra_calculate_metric_callback, tokenizer=tokenizer)
metric, all_label, all_logits  = evaluator.eval(0)

2023-10-18 15:17:41,592 <DEBUG> TextDataset: ⏳ Loading VALIDATION dataset ...
2023-10-18 15:17:41,593 <INFO> TextDataset: Model max length: 4096


Tokenize input texts:   0%|          | 0/4 [00:00<?, ?it/s]

2023-10-18 15:17:41,640 <DEBUG> TextDataset: ⌛ Loading VALIDATION data takes 0.05 sec.
2023-10-18 15:17:41,640 <INFO> TextDataset: Total data: 4
2023-10-18 15:17:41,641 <INFO> TextDataset: Max length of input: 795
2023-10-18 15:17:41,641 <INFO> TextDataset: Max length of label: -1


VALIDATION:   0%|          | 0/2 [00:00<?, ?batch/s]

VALIDATION:   0%|          | 0/2 [00:00<?, ?batch/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
from pprint import pp
print(metric)
for i in range(4):
    print("-----label------")
    pp(all_label[i])
    print("-----pred------")
    for text in all_logits[i]:
        print(text)
        print('--------------------------')
    print('########################################')

{'self-bleu1': 0.40526503186515067, 'self-bleu2': 0.5724364766829015, 'self-bleu3': 0.6763363723189562, 'self-bleu4': 0.7455375995634501}
-----label------
('\n'
 '徐怀钰近期参加综艺节目《乘风》复出，享受舞台找回自我。\n'
 '徐怀钰的经历充满波折，家庭贫困、爷爷的心理问题，这些年一直在努力付出。虽然在《乘风》的舞台上复出效果不理想，但我们也看到了徐怀钰逐渐找回自我，这比比赛的输赢更重要。在《乘风》中，她与秋瓷炫、黄丽玲等人成为了好友，并携手合作。虽然她没有王心凌的机遇，但《乘风》也为她提供了一个重要的机会。她因一公被抨击而感到难过，好在有Ella等人的安慰与支持，让她挺过了难关。五公时，慢热的徐怀钰终于找到了感觉，秋瓷炫也为她的变化感到高兴，看出她开始享受舞台了。这次，徐怀钰主动和黄丽玲合作，无疑将带来许多火花。\n'
 '作为徐怀钰复出的舞台，《乘风》让徐怀钰找回终于找回了自我。让我们一起期待徐怀钰更多精彩表现！')
-----pred------
在《乘风》后，徐怀钰的家庭贫困和爷爷的心理问题引发广泛关注。但是，她在节目中却逐渐找回了自我，与秋瓷炫等人成为了好友，并和黄丽玲合作了一首歌曲。
尽管徐怀钰并非王心凌，但在乘风中她展现出了自我找回的意义，获得了重拾表演的乐趣。她的经历让人心疼，从小家庭贫困、爷爷对她不好，导致她一直有着心理创伤。尽管努力复出，但效果并不理想。直到参加乘风时，因为路灯下的小姑娘被抨击而难过，幸亏Ella等人的支持才让她挺过去。第五届公演中，徐怀钰终于找到了感觉，展现出了自己的特色。秋瓷炫对徐怀钰的变化感到欣喜，认为她开始享受舞台了。此次与黄丽玲合作更是让人期待，两位实力唱将的合作必将产生更多火花。
徐怀钰的复出也许对某些人来说是意料之中，但她的成功却超乎所有人的预期。无论如何，对于粉丝来说，乘风只是一个综艺，能帮徐怀钰找回自我，比什么输赢都重要。让我们一起期待徐怀钰在未来更多的表现吧！你是否赞同徐怀钰的观点？欢迎留言评论分享你的想法！
--------------------------
徐怀钰参加综艺节目《乘风》后，引起广泛关注。她曾经历了家庭贫困和爷爷的心理问题，复出时效